In [81]:
import mysql.connector
from pprint import pprint
import spacy
nlp = spacy.load('en_core_web_sm')

mydb = mysql.connector.connect(
    host="35.226.133.103",
    user="root",
    passwd="keonp2",
    db="project_db"
)

cur = mydb.cursor()
courseNo = 411
courseName = 'Database Systems'
instructorId = 71

In [82]:
def get_prof_rankings_for_course(cur, courseNo, courseName):
    
    def normalize_score(min_s, max_s, score):
        return (score - min_s) / (max_s - min_s)
    
    # 1. get course and its text data
    q = '''
        SELECT courseDesc
        FROM csCourse
        WHERE courseNo = %s AND courseName = %s;
    '''
    
    cur.execute(q, (courseNo, courseName))
    res = cur.fetchone()
    courseDesc = res[0]
    
    course_text = courseName + ", " + courseDesc
    course_text_nlp = nlp(course_text)
    
    # 2. get all instructors' research interests 
    q = '''
        SELECT instructorId, instructorName, researchInterests
        FROM csInstructor;
    '''
    
    cur.execute(q)
    res = cur.fetchall()
    
    prof_research_dict = {}
    prof_id_dict = {}
    for r in res:
        prof_research_dict[r[0]] = nlp(r[2])
        prof_id_dict[r[0]] = str(r[1])
        
    # 3. get average GPA for instructors who have taught the course
    q = '''
        SELECT instructorId, (((SUM(aPlus) * 4) + (SUM(a) * 4) + (SUM(aMinus) * 3.67) + (SUM(bPlus) * 3.33) + (SUM(b) * 3) + (SUM(bMinus) * 2.67) + (SUM(cPlus) * 2.33) + (SUM(c) * 2) + (SUM(cMinus) * 1.67) + (SUM(dPlus) * 1.33) + (SUM(d) * 1) + (SUM(dMinus) * 0.67) + (SUM(f) * 0)) / (SUM(aPlus) + SUM(a) + SUM(aMinus) + SUM(bPlus) + SUM(b) + SUM(bMinus) + SUM(cPlus) + SUM(c) + SUM(cMinus) + SUM(dPlus) + SUM(d) + SUM(dMinus) + SUM(f))) as averageGPA
        FROM csGrade LEFT JOIN csInstructor ON csInstructor.instructorId = csGrade.primaryInstructor
        WHERE csGrade.courseNo = %s AND csGrade.courseName = %s
        GROUP BY csInstructor.instructorId;
    '''
    
    cur.execute(q, (courseNo, courseName))
    res = cur.fetchall()
    
    prof_avg_dict = {}
    for r in res:
        prof_avg_dict[r[0]] = float(r[1])
    base_score = float(min([r[1] for r in res]))
    
    sim_score_weight = 0.1
    
    max_s = 0
    min_s = 0
    prof_total_scores = []
    all_scores = []
    
    for prof in prof_research_dict:
        sim_score = 0
        if prof_research_dict[prof].vector_norm:
            sim_score = float(course_text_nlp.similarity(prof_research_dict[prof]))

        gpa_score = base_score
        if prof in prof_avg_dict:
            gpa_score = prof_avg_dict[prof]
            
        total_score = sim_score * sim_score_weight + gpa_score
            
        prof_data = { 
            "instructorId": prof, 
            "instructorName": prof_id_dict[prof], 
            "researchInterests": prof_research_dict[prof].text, 
            "score": total_score
        }

        all_scores.append(total_score)
        prof_total_scores.append(prof_data)
        
    max_s = max(all_scores)
    min_s = min(all_scores)
    for prof_data in prof_total_scores:
        prof_data["score"] = normalize_score(min_s, max_s, prof_data["score"])
    sorted_prof_scores = sorted(prof_total_scores, key= lambda p : p["score"], reverse=True)
    return sorted_prof_scores[:5]

In [83]:
def get_course_rankings_for_prof(cur, instructorId):
    
    def normalize_score(min_s, max_s, score):
        return (score - min_s) / (max_s - min_s)
    
    # 1. get prof and his/her research interests
    q = '''
        SELECT instructorId, instructorName, researchInterests, COUNT(*)
        FROM csInstructor LEFT JOIN csGrade ON csInstructor.instructorId = csGrade.primaryInstructor
        WHERE instructorId = %s;
    '''
    
    cur.execute(q, (instructorId,))
    instructorId, instructorName, researchInterests, termsTaught = cur.fetchone()
    instructor_text_nlp = nlp(researchInterests)
    
    print(instructorId, instructorName, researchInterests, termsTaught)
    
    if not instructor_text_nlp.vector_norm:
        return [] # cant match prof to courses if prof research interests is empty
    
    # 2. get all courses and their text data
    q = '''
        SELECT courseNo, courseName, courseDesc
        FROM csCourse;
    '''
    
    cur.execute(q)
    res = cur.fetchall()
    
    course_desc_dict = {}
    for r in res:
        course_desc_dict[(r[0], r[1])] = nlp(r[2])
        
    # 3. get courses that the instructor has taught before
    q = '''
        SELECT courseNo, courseName, COUNT(*)
        FROM csGrade 
        WHERE primaryInstructor = %s
        GROUP BY courseNo, courseName;
    '''
    
    cur.execute(q, (instructorId,))
    res = cur.fetchall()
    
    course_taught_dict = {}
    for r in res:
        course_taught_dict[(r[0], r[1])] = r[2] / termsTaught
    
    all_scores = []
    course_total_scores = []
    taught_weight = 0.1
    
    for course in course_desc_dict:
        courseNo, courseName = course
        
        score = 0
        if course in course_taught_dict:
            score += taught_weight * course_taught_dict[course]
        
        if course_desc_dict[course].vector_norm:
            score += instructor_text_nlp.similarity(course_desc_dict[course])
            
        course_data = {
            "courseNo": courseNo,
            "courseName": courseName,
            "courseDesc": course_desc_dict[course].text,
            "score": score
        }
        
        all_scores.append(score)
        course_total_scores.append(course_data)
        
    max_s = max(all_scores)
    min_s = min(all_scores)
    for course_data in course_total_scores:
        course_data["score"] = normalize_score(min_s, max_s, course_data["score"])
    
    sorted_course_scores = sorted(course_total_scores, key = lambda c : c["score"], reverse=True)
    return sorted_course_scores[:10]

In [84]:
# for a given course, get and rank instructors for that course (get top 5)
results = get_prof_rankings_for_course(cur, courseNo, courseName)
pprint(results)

c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the la

c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the la

c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the la

c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the la

[{'instructorId': 71,
  'instructorName': 'Abdussalam Alawini',
  'researchInterests': 'Active Learning in Large Classrooms, Teamwork and '
                       'Collaboration, Computer-Based Assessment, '
                       'Instructional Technologies, Data Provenance, '
                       'Scientific Data Management, Data Citation, Workflow '
                       'Management, Machine Learning',
  'score': 1.0},
 {'instructorId': 12,
  'instructorName': 'Ryan Cunningham',
  'researchInterests': '',
  'score': 0.8067530466122067},
 {'instructorId': 166,
  'instructorName': 'Timothy Weninger',
  'researchInterests': '',
  'score': 0.7588256558517477},
 {'instructorId': 13,
  'instructorName': 'Aditya Parameswaran',
  'researchInterests': '',
  'score': 0.7002064978636077},
 {'instructorId': 61,
  'instructorName': 'Saurabh Sinha',
  'researchInterests': 'Bioinformatics, Genomics, Modeling, Sequence Analysis, '
                       'Machine Learning, Probabilistic Methods, 

c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the la

In [85]:
results = get_course_rankings_for_prof(cur, instructorId)
pprint(results)

71 Abdussalam Alawini Active Learning in Large Classrooms, Teamwork and Collaboration, Computer-Based Assessment, Instructional Technologies, Data Provenance, Scientific Data Management, Data Citation, Workflow Management, Machine Learning 2


c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the la

c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the la

c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the la

c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the la

c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the la

c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the la

[{'courseDesc': 'NP-completeness, design and analysis techniques, '
                'approximation algorithms, randomized algorithms, '
                'combinatorial optimization, linear programming. Intended for '
                'graduate students in Computer Science. ',
  'courseName': 'Algorithms',
  'courseNo': 573,
  'score': 1.0},
 {'courseDesc': 'Advanced concepts, models, and algorithms in information '
                'retrieval and major recent developments in the field, '
                'including historical milestones in information retrieval '
                'research, evaluation methodology, vector space retrieval '
                'model, probabilistic retrieval models, learning to rank '
                'algorithms, probabilistic topic models, information retrieval '
                'systems, text analytics, and topics of research frontiers in '
                'information retrieval. ',
  'courseName': 'Advanced Information Retrieval',
  'courseNo': 510,
  'score':

c:\program files\python37\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
